# Creating and Deploying Agents Programmatically
Amazon Bedrock provides an SDK for developers to **create, configure, and deploy AI agents** programmatically. Agents act as intermediaries between **users and foundation models (FMs)** to handle tasks and process requests.

This guide demonstrates how to create an **HR agent** using the **boto3 Python SDK**.

## Step 1: Create an IAM Policy for the Agent
Amazon Bedrock **Agents** need an **IAM policy** that grants the ability to invoke the foundation model.

In [ ]:
import boto3
import json

iam_client = boto3.client("iam")

bedrock_agent_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                "arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName="hr-agent-bedrock-policy",
    PolicyDocument=bedrock_policy_json
)

- Additional optional agent policies:
  - API Schema definitions policy - agent uses API Schema definitions for the action group
  - Amazon Bedrock Knowledge Base policy - agent connects to a Amazon Bedrock Knowledge Base, you need to create an IAM Policy
  - Amazon Bedrock Guardrail policy - agent connects to a Amazon Bedrock Guardrail, you need to create an IAM Policy

## Step 2: Create an IAM Role for the Agent
An IAM role must be created with Amazon Bedrock as the principal.

In [ ]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
            "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

agent_role_name = "AmazonBedrockExecutionRoleForAgents_hr-agent"

assume_role_policy_document_json = json.dumps(assume_role_policy_document)

agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

## Step 3: Attaching the created policies to the role
Attach the created policies to the IAM role.

In [ ]:
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

# Optionally, attach additional policies (S3, Knowledge Bases, Guardrails)

## Step 4: Creating the Amazon Bedrock agent
Now that the IAM role is created, use the boto3 SDK to create the HR agent.

In [ ]:
bedrock_agent_client = boto3.client('bedrock-agent')

response = bedrock_agent_client.create_agent(
    agentName="Human-Resource-Agent",
    description="This agent helps employees understand company policies and request vacation time.",
    foundationModel="anthropic.claude-3-sonnet-20240229-v1:0",
    agentResourceRoleArn=agent_role["Role"]["Arn"],
    idleSessionTTLInSeconds=600,
    instruction="""You are a friendly agent that answers questions about the 
    company's HR policies and helps employees request vacation time off.
    You ALWAYS reply politely and concisely, using ONLY the available information 
    in the company_policies KNOWLEDGE_BASE or the vacationHandler ACTION_GROUP.
    Introduce yourself as the 'HR AI Assistant' and confirm the employee's ID 
    before providing any details."""
)

## Step 5: Preparing the agent
After creating the agent, use prepare_agent to enable testing.

In [ ]:
agent_id = response['agent']['agentId']

response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)

## **Key Takeaways**
- This guide demonstrated how to:
  - Create an IAM Policy for the agent.
  - Set up an IAM Role for execution.
  - Attach necessary policies to the role.
  - Create and configure an agent using boto3.
  - Prepare the agent for invocation and testing.